In [21]:
# import necessary packages
import numpy as np
import scipy as sci
import matplotlib as plt

In [22]:
class QuantumRegister:
    def __init__(self, N):
        # Initialize column vector
        self.N = N
        self.psi = 1 / np.sqrt(2**self.N) * np.zeros((2**N, 1), dtype=complex)
        # Set the amplitude of the first state
        self.psi[0] = 1
    
    def get_register(self):
        # Method to return the quantum state
        return self.psi
    
    def get_probabilities(self):
        return np.abs(self.psi)**2

    def measure_state(self):
        probabilities = np.abs(self.psi)**2
        # select a basis state randomly according to the probabilities
        measurement = np.random.choice(np.arange(len(self.psi)), p=probabilities.ravel())
        return "| " + np.binary_repr(measurement, width = self.N) + " >"

    def target_state(self):
        target = [1] * (self.N - 1)
        target.append(0)
        targetPrint = ''.join(str(x) for x in target)
        return "| " + targetPrint + " >"

    def ApplyGate(self,gate):
        self.psi = np.matmul(gate,self.psi)

    

In [23]:
class Hadamard:   
    @staticmethod
    def HadamardTransform(N):
        # define the initial Hadamard matrix
        hadamard_initial = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

        # number of iterations for the transformation
        num_iterations = N - 1

        # initialize the resulting tensor product matrix
        hadamard_result = hadamard_initial

        # perform tensor product iteratively
        for i in range(num_iterations):
            hadamard_result = np.kron(hadamard_initial, hadamard_result)

        return hadamard_result
    
    @staticmethod
    def ApplyHadamard(QuantReg):
        N = QuantReg.N
        HadamardGate = Hadamard.HadamardTransform(N)
        QuantReg.ApplyGate(HadamardGate)

In [24]:
class Oracle:
    @staticmethod
    def OracleGate(N,target):
        #initialise oracle matric with target
        oracleInitalise = np.eye(2**N)
        oracleInitalise[target -1][target-1]=-1
        
        return oracleInitalise
    
    @staticmethod
    def ApplyOracle(QuantReg):
        N = QuantReg.N
        target = (2**N) - 1
        OracleOperator = Oracle.OracleGate(N,target)
        QuantReg.ApplyGate(OracleOperator)

In [25]:
class Diffusion:
    @staticmethod
    def DiffusionOperator(N):
        # Define superposition state
        s = np.zeros((2**N, 1), dtype=complex) #/ np.sqrt(2**N)
        s[0][0]=1
        # Compute |s><s|
        ss_dagger = np.dot(s,s.T.conj())
        # ID Matric
        identity = np.eye(2**N)
        # Diffusion Operator
        diffusion =  (2 * ss_dagger) - identity 
        return diffusion
    
    @staticmethod
    def ApplyDiffusion(QuantReg):
        N = QuantReg.N
        DiffusionGate = Diffusion.DiffusionOperator(N)
        QuantReg.ApplyGate(DiffusionGate)

In [26]:
class Grover:
    @staticmethod
    def ApplyAlgorithm(QuantReg):
        N = QuantReg.N
        IdealLoop = ((np.pi/4) * np.sqrt(2**N)).astype(int)
        Hadamard.ApplyHadamard(QuantReg)
        for i in range(IdealLoop):
            Oracle.ApplyOracle(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
            Diffusion.ApplyDiffusion(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
        

In [27]:
"""
TESTING BLOCK
"""

"""creates the quantum register """
N=3
target = 7
quantum_Register1 = QuantumRegister(N)
print(quantum_Register1.get_register())
print("Current State: ",quantum_Register1.measure_state())
print("Target State:  ",quantum_Register1.target_state())

"""operating on the register using the gates"""
#Applying the Hadamard
Hcheck = Hadamard.HadamardTransform(N)
#print(Hcheck)

#Applying the Diffusion Operator
diffuse =  Diffusion.DiffusionOperator(N)
#print(diffuse)

#Applying the Oracle
Ocheck = Oracle.OracleGate(N,target)
#print(Ocheck)

"""Trying to work Grovers"""
Hadamard.ApplyHadamard(quantum_Register1)
for i in range(2):
    Oracle.ApplyOracle(quantum_Register1)
    Hadamard.ApplyHadamard(quantum_Register1)
    Diffusion.ApplyDiffusion(quantum_Register1)
    Hadamard.ApplyHadamard(quantum_Register1)
print(quantum_Register1.get_probabilities())
print(quantum_Register1.measure_state())


[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
Current State:  | 000 >
Target State:   | 110 >
[[0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.9453125]
 [0.0078125]]
| 110 >


In [28]:
qr = QuantumRegister(3)
print("Begin State: ", qr.measure_state())
print(qr.get_probabilities())


Grover.ApplyAlgorithm(qr)
print("After Algorithm: ", qr.measure_state())
print(qr.get_probabilities())

Begin State:  | 000 >
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
After Algorithm:  | 110 >
[[0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.9453125]
 [0.0078125]]


In [30]:
qrBig = QuantumRegister(12)
print("Begin State: ", qrBig.measure_state())
print(qrBig.get_probabilities())


Grover.ApplyAlgorithm(qrBig)
print("After Algorithm: ", qrBig.measure_state())
print(qrBig.get_probabilities())

Begin State:  | 000000000000 >
[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


KeyboardInterrupt: 